In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset
from random import shuffle
import spacy
import re

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
dataset = load_dataset("google-research-datasets/paws", "labeled_final")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
def tokenize_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z\s]', '', sentence)
    doc = nlp(sentence)
    tokens = [
        token.lemma_  # Lemmatized token
        for token in doc
        if token.is_alpha and not token.is_stop  # Keep only alphabetic and non-stopwords
    ]
    return tokens


# Prepare TaggedDocument data
tagged_data = []
for idx, example in enumerate(dataset['train']):
    tagged_data.append(TaggedDocument(words=tokenize_sentence(example['sentence1']), tags=[f"s1_{idx}"]))
    tagged_data.append(TaggedDocument(words=tokenize_sentence(example['sentence2']), tags=[f"s2_{idx}"]))

# Shuffle the data
shuffle(tagged_data)

In [ ]:
vector_size = 300
doc2vec_model = Doc2Vec(tagged_data, vector_size=vector_size, window=2, min_count=1, workers=4, epochs=40)

# Infer embeddings for similarity
similarities = []
labels = []

for example in dataset['train']:
    sentence1_vector = doc2vec_model.infer_vector(tokenize_sentence(example['sentence1']))
    sentence2_vector = doc2vec_model.infer_vector(tokenize_sentence(example['sentence2']))
    similarity = cosine_similarity([sentence1_vector], [sentence2_vector])[0][0]
    similarities.append(similarity)
    labels.append(example['label'])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Prepare for classification
X = np.array(similarities).reshape(-1, 1)
y = np.array(labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define parameter grid for Logistic Regression
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'penalty': ['l1', 'l2'],        # Regularization type
    'solver': ['liblinear']         # Solver for l1 and l2
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    LogisticRegression(),
    param_grid,
    scoring='accuracy',  # Use accuracy as the evaluation metric
    cv=7,                # 5-fold cross-validation
    verbose=1            # Print progress
)

# Perform the grid search on training data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Use the best model to make predictions on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model on the test set
print("\nTest Set Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Fitting 7 folds for each of 10 candidates, totalling 70 fits
Best Parameters: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Cross-Validation Accuracy: 0.5882084981068498

Test Set Accuracy: 0.590932091893533

Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.89      0.71      5549
           1       0.60      0.21      0.31      4332

    accuracy                           0.59      9881
   macro avg       0.59      0.55      0.51      9881
weighted avg       0.59      0.59      0.53      9881

